In [1]:
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns; sns.set();

In [14]:
df = pd.read_csv('C:\\Users\\ryan\\Desktop\\Thinkful DS Sample Data - Main Course\\Unit 3\\breast_cancer_data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
#Attribute Information:
#1) ID number
#2) Diagnosis (M = malignant, B = benign)
#-3-32.Ten real-valued features are computed for each cell nucleus:
#a) radius (mean of distances from center to points on the perimeter)
#b) texture (standard deviation of gray-scale values)
#c) perimeter
#d) area
#e) smoothness (local variation in radius lengths)
#f) compactness (perimeter^2 / area - 1.0)
#g). concavity (severity of concave portions of the contour)
#h). concave points (number of concave portions of the contour)
#i). symmetry
#j). fractal dimension ("coastline approximation" - 1)

In [15]:
#This dataset has 10 'unique' features, and three versions of these features (mean, SE, and 'worst', meaning the avg of the 
#3 largest... see dataset description here: https://www.kaggle.com/uciml/breast-cancer-wisconsin-data)

#The point is, the data should show some multicolinearity..meaning that our ridge regression should come in handy here, 
#and that we could see some higher than usual coefficients in the regular logistic regression model

#Convert diagnosis to binary from M (malignant) and B (benign)

df['diagnosis'] = df['diagnosis'].apply(lambda x: 1 if x == 'M' else 0)

In [16]:
df.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [19]:
#print(df['Unnamed: 32'].unique())
#All values in this column are NAN, so drop it, and also drop the ID column
df.drop('Unnamed: 32', axis = 1, inplace = True)
df.drop('id', axis = 1, inplace = True)

In [20]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [139]:
from sklearn.linear_model import LogisticRegression

#REGULAR LOGISTIC REGRESSION

predictors = df.ix[:, 1:]
targets = df['diagnosis']

#Set the C value very high such that there is essentially no regularization taking place (C set to very high value)
lr = LogisticRegression(C = 1e9)

# Fit the model.
fit = lr.fit(predictors, targets)

print('\n Overall Percentage Accuracy')
print(lr.score(predictors, targets))

X = predictors
y = targets


from sklearn.model_selection import KFold

test_no = 1
kf = KFold(n_splits=5) 

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #Use model defined above
    predicted = lr.fit(X_train, y_train).predict(X_test)
    actual = y_test
    print('Accuracy: Fold', test_no)
    print(lr.score(X_test, y_test))
    test_no = test_no + 1

#Store the parameter estimates.
origparams = np.append(lr.coef_, lr.intercept_)
print(origparams)


 Overall Percentage Accuracy
0.966608084359
Accuracy: Fold 1
0.912280701754
Accuracy: Fold 2
0.956140350877
Accuracy: Fold 3
0.973684210526
Accuracy: Fold 4
0.964912280702
Accuracy: Fold 5
0.946902654867
[ -2.45670877e+00   6.33033290e-02  -2.93359288e-02   6.27779905e-03
   1.57600177e-01   4.96699759e-01   6.96998896e-01   3.53491799e-01
   1.98239804e-01   3.74561653e-02  -3.23224050e-02  -1.57427141e+00
  -3.48563541e-01   1.36506864e-01   1.65129875e-02   4.12591558e-02
   8.22908235e-02   4.28349255e-02   4.85094635e-02  -2.06870102e-03
  -1.91551463e+00   2.91216265e-01   3.00362342e-01   2.58773616e-02
   3.04943717e-01   1.59087259e+00   2.00579979e+00   7.14479945e-01
   6.87200637e-01   1.67541447e-01  -4.40542725e-01]


In [ ]:
#In your report, evaluate all three models and decide on your best. Be clear about the decisions you made that led 
#to these models (feature selection, regularization parameter selection, model evaluation criteria) and why you 
#think that particular model is the best of the three. Also reflect on the strengths and limitations of regression 
#as a modeling approach. Were there things you couldn't do but you wish you could have done?

In [153]:
#Now try ridge regression to see if adding a regularization penalty can help improve accuracy

#RIDGE REGRESSION (L2)

from sklearn.linear_model import LogisticRegressionCV

#Set the C value lower such that the regularization does have an impact
lr = LogisticRegressionCV(penalty='l2', solver = 'liblinear', cv = 5)

# Fit the model.
fit = lr.fit(predictors, targets)

print('\n Overall Percentage Accuracy')
print(lr.score(predictors, targets))

X = predictors
y = targets


from sklearn.model_selection import KFold

test_no = 1
kf = KFold(n_splits=5) 

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #Use model defined above
    predicted = lr.fit(X_train, y_train).predict(X_test)
    actual = y_test
    print('Accuracy: Fold', test_no)
    print(lr.score(X_test, y_test))
    test_no = test_no + 1

#Store the parameter estimates.
cvparams = np.append(lr.coef_, lr.intercept_)
print(cvparams)


 Overall Percentage Accuracy
0.966608084359
Accuracy: Fold 1
0.964912280702
Accuracy: Fold 2
0.947368421053
Accuracy: Fold 3
0.973684210526
Accuracy: Fold 4
0.973684210526
Accuracy: Fold 5
0.946902654867
[ -2.29151998e+00   5.65773698e-02  -4.34668809e-02   5.54213220e-03
   1.35592165e-01   4.59717138e-01   6.35770653e-01   3.12964237e-01
   1.70935144e-01   3.42258506e-02  -4.39905926e-02  -1.40248453e+00
  -4.08637731e-01   1.35792958e-01   1.38725507e-02   5.10724766e-02
   8.62945215e-02   3.79099124e-02   4.38306885e-02   4.72105356e-04
  -1.90722494e+00   2.77798156e-01   3.02395521e-01   2.54423956e-02
   2.60972635e-01   1.47614934e+00   1.83398327e+00   6.29868590e-01
   5.97596184e-01   1.54065529e-01  -4.11262277e-01]


In [154]:
#LASSO REGRESSION (L1)

#LOGISTICREGRESSIONCV is supposed to find the optimal C value

from sklearn.linear_model import LogisticRegressionCV

#Set the C value lower such that the regularization does have an impact
lr = LogisticRegressionCV(penalty='l1', solver = 'liblinear', cv = 5)

# Fit the model.
fit = lr.fit(predictors, targets)

print('\n Overall Percentage Accuracy')
print(lr.score(predictors, targets))

X = predictors
y = targets


from sklearn.model_selection import KFold

test_no = 1
kf = KFold(n_splits=5) 

for train_index, test_index in kf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    #Use model defined above
    predicted = lr.fit(X_train, y_train).predict(X_test)
    actual = y_test
    print('Accuracy: Fold', test_no)
    print(lr.score(X_test, y_test))
    test_no = test_no + 1

#Store the parameter estimates.
cvparams = np.append(lr.coef_, lr.intercept_)
print(cvparams)


 Overall Percentage Accuracy
0.992970123023
Accuracy: Fold 1
0.921052631579
Accuracy: Fold 2
0.938596491228
Accuracy: Fold 3
0.964912280702
Accuracy: Fold 4
0.982456140351
Accuracy: Fold 5
0.973451327434
[ -2.17839853e+00   2.79088872e-01  -2.82118068e-01   2.51822444e-02
   0.00000000e+00  -1.92639781e+01   7.72919698e+00   6.18783271e+01
  -1.09138801e+01   0.00000000e+00   2.52539090e+00  -1.25218807e+00
  -2.45863940e-01   1.78981848e-01   0.00000000e+00  -2.65447895e+01
  -3.71976706e+01   0.00000000e+00   0.00000000e+00   0.00000000e+00
  -9.02909783e-02   2.47755562e-01   7.56253983e-02   1.94544392e-02
   4.93670131e+01  -3.78383229e+00   1.00519199e+01   4.79731262e+01
   7.24526574e+00   0.00000000e+00  -1.12551022e+01]


In [ ]:
#I chose to use the LogisiticRegressionCV function because it has a built-in solver that finds the optimal C-value
#to use for the regularization parameter 

#Overall, I would opt to use the ridge regresssion (L2) logisticregressioncv function because although the overall model accuracy 
#was not the highest (96.6% vs 98.6% for LASSO), it performed more consistently on the cross-validation tests with accuracy
#ranging from ~95%-97% while the LASSO had less consistent perforamance (accuracy ranged from 92%-98%)

In [ ]:
#Could also fit the data to a random forest to determine which are the most important features, and then re-run the models